In [1]:
import gymnasium as gym
import os
import torch
from stable_baselines3 import SAC
import random
import gymnasium as gym
from PIL import Image
import time
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete
import numpy as np
import yaml
from torchvision import transforms
from stable_baselines3.common.vec_env import DummyVecEnv



In [14]:
env_name = "MountainCarContinuous-v0"
#env_name = "MountainCar-v0"
env= gym.make(env_name,render_mode ='rgb_array')
env = PixelObservationWrapper(env)

In [29]:
obs,_ = env.reset()
frame = obs['pixels']
val_transforms = transforms.Compose([transforms.ToTensor(),
        #AddGaussianNoise(self.mean, self.std),
        transforms.Resize((64,64))
        ])
frame = val_transforms(frame) #(c,h,w)
frame = torch.unsqueeze(frame, 0)

/home/max/miniconda3/envs/simclr/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [30]:
print(frame.shape)

torch.Size([1, 3, 64, 64])


In [54]:
h, _, enc, zj = model(frame, frame)
print(h.shape, z.shape)

torch.Size([1, 512]) torch.Size([1, 64])


In [52]:
enc[0]-zj[0]


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       grad_fn=<SubBackward0>)

In [55]:
d = np.array([tensor.detach().cpu().numpy() for tensor in enc])

In [59]:
d[0].shape

(64,)

In [2]:
#build the wrapper
class VAE_ENC(ObservationWrapper):
    def __init__(self, env, vae,
                 mean=0,std=0.1,
                 size=(64,64)):
        super().__init__(env)
        #new obs space
        self.observation_space = Box(shape=(64,), low=-np.inf, high=np.inf)

        self.vae = vae
        #transforms
        self.mean = mean
        self.std = std
        self.size = size
        
        
        
    def observation(self, obs):
        #get frame
        #print(obs)
        frame = obs['pixels']#.to('cuda')
        #transform for VAE
        val_transforms = transforms.Compose([transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),
        #AddGaussianNoise(self.mean, self.std),
        transforms.Resize(self.size),
        #transforms.Grayscale(),
        #transforms.Normalize(self.mean, self.std),
        ])
        frame = val_transforms(frame).to('cpu') #(c,h,w)
        frame = torch.unsqueeze(frame, 0)#.to(self.device) #make it (1,c,h,w)
        res_net,res_net1,enc,enc_1 = self.vae(frame, frame)  # takes two pictures, bc loss is comparrison
        # outputs resnet for both pictures, and enc in feature dim
        enc = np.array([tensor.detach().cpu().numpy() for tensor in enc])
        enc = np.array(enc[0])
        return enc

In [3]:
# Load
if torch.cuda.is_available():
    device = torch.device('cpu')#("cuda")
else:
    device = torch.device("cpu")
PATH = "third_lunar_64.pth" #"model_architecture.pth"#"model_mountain_car.pth"
#device = torch.device('gpu')
model = torch.load(PATH, map_location=device)

env_name = "LunarLander-v2" #"MountainCarContinuous-v0"

env = gym.make("LunarLander-v2",
                continuous= True,
                gravity= -10.0,
                enable_wind= False,
                wind_power= 15.0,
                turbulence_power= 1.5,
                render_mode ='rgb_array')

#env_name = "MountainCar-v0"
#env= gym.make(env_name,render_mode ='rgb_array')
env = PixelObservationWrapper(env)
env = VAE_ENC(env, model)
env = FrameStack(env, num_stack=2)
env = Monitor(env)

In [4]:
obs, info=env.reset()

/home/max/miniconda3/envs/simclr/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [5]:
obs.shape

(2, 64)

In [6]:
RL_train_steps = 500000 #episode max_len=999 -> ~50 episodes
agent_name = "simclr64_2_agent_data_SAC"
env_name = "LunarLander-v2"
seed = 42
i=2
models_dir = f"RLmodels/"+env_name
logdir = f"RLlogs/"+env_name

callback = CheckpointCallback(save_freq=10000, save_path=models_dir+str(i))  

In [7]:
agent = SAC(
    env = env,
    policy= 'MlpPolicy',
    batch_size= 128,
    learning_rate=0.0003,
    buffer_size= 100000, #1000000,
    ent_coef='auto',
    gamma= 0.99,
    tau= 0.01,
    train_freq= 1,
    gradient_steps= 1,
    learning_starts= 10000,
    policy_kwargs= dict(net_arch=[400,300]),#dict(net_arch=[400, 300])
    tensorboard_log=logdir+str(i)
)

In [8]:
agent.learn(total_timesteps=RL_train_steps,callback=callback,tb_log_name=agent_name)


In [13]:
env


<Monitor<FrameStack<VAE_ENC<PixelObservationWrapper<TimeLimit<OrderEnforcing<PassiveEnvChecker<LunarLander<LunarLander-v2>>>>>>>>>

In [12]:
agent = SAC.load('RLmodels/LunarLander-v2/rl_model_480000_steps')


In [14]:
save_path='agent_data'
data_name='simclr64SAC_after_third_train'
num_of_episodes=100
i=0
for episode in range(num_of_episodes):
    observation, info = env.reset()
    done = False
    while not done: 
        action, _states = agent.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        #state, reward, terminated, truncated, info = env.step(action)
        #print(terminated, truncated)
        if terminated:
            done = True
        if truncated:
            done = True
        current_frame = env.render()

        i+=1
        im = Image.fromarray(np.array(current_frame))
        im.save(save_path+'/'+data_name+'_'+str(i)+'.jpeg')

array([-0.16585077,  0.76528394], dtype=float32)

In [5]:
data_name='initial_random_action'
save_path='Data/lunar-lander/'+data_name+'/'

env = gym.make("LunarLander-v2",
                continuous= True,
                gravity= -10.0,
                enable_wind= False,
                wind_power= 15.0,
                turbulence_power= 1.5,
                render_mode ='rgb_array')

num_of_episodes=100
i=0
for episode in range(num_of_episodes):
    observation, info = env.reset()
    done = False
    while not done: 
        action= env.env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        #state, reward, terminated, truncated, info = env.step(action)
        #print(terminated, truncated)
        if terminated:
            done = True
        if truncated:
            done = True
        current_frame = env.render()

        i+=1
        im = Image.fromarray(np.array(current_frame))
        im.save(save_path+'/'+data_name+'_'+str(i)+'.jpeg')

In [9]:
agent = SAC(   
    policy='MlpPolicy',
    env=env,
    seed=seed,
    learning_rate=0.0003,
    buffer_size= 50000,
    batch_size= 512,
    ent_coef= 0.1,
    train_freq= 32,
    gradient_steps= 32,
    gamma= 0.9999,
    tau= 0.01,
    learning_starts= 0,
    use_sde= True,
    policy_kwargs= dict(log_std_init=-3.67, net_arch=[64, 64]),
    tensorboard_log=logdir)